In [1]:
import numpy as np
from uuid import uuid4
from sklearn.manifold import TSNE
from sklearn.cluster import AffinityPropagation
import json
from copy import copy, deepcopy

import pprint

In [2]:
pp = pprint.PrettyPrinter().pprint

In [3]:
# file_name = 'labeled_sentiments.json'
file_name = 'output_1000.json'
with open(file_name, 'r') as f:
    lines = f.readlines()
for line in lines:
    line = line.strip()

print(len(lines))
embedding_data = [json.loads(line) for line in lines] 
# embedding_data[0]

1002


In [4]:
def run_tsne(x):
    """
    Reduce the dimension of the input vector

    Arg:
        x (numpy.array): a list of lists containing vectors.

    Retrun:
        (numpy.array): reduced dimension
    """
    low_dim_x = TSNE(
        n_components=2,
        learning_rate=200,
        perplexity=30
    ).fit_transform(x)
    return low_dim_x

def reduce_dimension(embedding_data):
    """
    Extracts the hight dimension vectors from the data, 
    run the dimension reducsion algorithm and add the 
    low dimension vectors to the original data
    """
    # get vectors
    embedding_data = copy(embedding_data)
    vect_list = []
    for e in embedding_data:
        vect_list.append(e.get('embedding'))

    # reduce dimension
    vect_array = np.array(vect_list)
    low_dim_embeddings = run_tsne(vect_array)
    
    # Merge back the low dimensions into the original data
    for i, item in enumerate(embedding_data):
        item['low_dim_embedding'] = list(low_dim_embeddings[i])
    return embedding_data

# Reducing dimension
data_w_low_dim = reduce_dimension(embedding_data)

In [5]:
pp(data_w_low_dim[:10])

[{'embedding': [0.08265981823205948,
                -0.003910927567631006,
                -0.09897520393133163,
                0.00018613849533721805,
                0.011625337414443493,
                -0.0034296722151339054,
                -0.046466533094644547,
                0.03300270438194275,
                -0.07402294874191284,
                -0.052507832646369934,
                0.00039215153083205223,
                -0.12111396342515945,
                0.030718045309185982,
                0.08059383183717728,
                0.005496548023074865,
                0.004270592704415321,
                0.007647651247680187,
                -0.018710695207118988,
                -0.017675651237368584,
                -0.0005542440921999514,
                -0.07012727111577988,
                -0.02079487033188343,
                -0.010672145523130894,
                -0.018036838620901108,
                -0.04745117574930191,
                -0.032987404614686966,

                -0.0954766571521759,
                0.04648680239915848,
                0.019555144011974335,
                -0.04176731035113335,
                0.03725830093026161,
                0.018663305789232254,
                -0.029656611382961273,
                -0.023571141064167023,
                0.02283105067908764,
                0.03546734154224396,
                -0.05167724937200546,
                -0.001748590380884707,
                -0.04487817361950874,
                -0.0655861422419548,
                -0.0318262055516243,
                -0.028368687257170677,
                -0.03088807314634323,
                0.044366102665662766,
                0.07644768804311752,
                -0.011321080848574638,
                0.048642002046108246,
                -0.06222936883568764,
                0.06133565306663513,
                -0.04903760179877281,
                -0.04436979442834854,
                -0.019488997757434845,
               

                -0.062043674290180206,
                0.009094806388020515,
                0.024553274735808372,
                -0.016726555302739143,
                0.0218045711517334,
                0.03685268759727478,
                0.05256199464201927,
                0.053418003022670746,
                -0.03858211264014244,
                -0.04964597523212433,
                -0.03825660049915314,
                0.02130989544093609,
                -0.047661587595939636,
                0.09492452442646027,
                0.008991732262074947,
                -0.06165504828095436,
                -0.02760806679725647,
                -0.003485205350443721,
                -0.004238583147525787,
                0.03218544274568558,
                0.004634313751012087,
                -0.054132770746946335,
                -0.007954786531627178,
                -0.07020308822393417,
                -0.010468186810612679,
                -0.04631218686699867,
           

                -0.05334194004535675,
                0.06727016717195511,
                0.05377935245633125,
                0.005618593655526638,
                0.014140961691737175,
                -0.009226460009813309,
                -0.05542697757482529,
                -0.020537326112389565,
                0.007549345027655363,
                -0.03754269331693649,
                0.014958223327994347,
                0.02706998772919178,
                0.017049692571163177,
                -0.03320536017417908,
                0.04222481697797775,
                0.004391438327729702,
                -0.003111552447080612,
                0.019260426983237267,
                -0.05008888989686966,
                0.07192184776067734,
                -0.06911224126815796,
                0.003993232734501362,
                -0.038991913199424744,
                0.015941016376018524,
                -0.008999492973089218,
                0.04908347874879837,
             

In [6]:
def ap_cluster(x):
    """
    Clusters data using affinity propagation algorithm.
    """
    clustering = AffinityPropagation(
        random_state=5, damping=0.95
    ).fit(x)

    cluster_labels = clustering.labels_
    cluster_centers = clustering.cluster_centers_
    return cluster_labels, cluster_centers

def cluster_data(data, coordinates_key=None):
    """
    cluster a listof objects with a 'coordinates' key
    
    Args
        coordinates_key : string, A name for the lookup key for the coordinates
        data : list[dicts], A list of objects that has a key for coordinates
        [
            {
                coordinates_key: [x1, x2, ...],
                ...
            },
            ...
        ]
        
    Returns
        Adds the clustering_info to each object in the input list of data
    """
    # Cluster data
    coordinates = []
    for item in data:
        coordinates.append(item[coordinates_key])
    cluster_labels, cluster_centers = ap_cluster(np.array(coordinates))
    
    # add clustering info to the data structure
    for i, item in enumerate(data):
        cluster_info = {}
        cluster_info['is_cluster_head'] = item[coordinates_key] in cluster_centers
        cluster_info['cluster_label'] = cluster_labels[i]
        item['cluster_info'] = cluster_info
        # TODO: remove this lines
        item['embedding'] = ''
#         item['text'] = ''
    
    # sort data
    result = sorted(
        data, key=lambda x:
            (x['cluster_info']['cluster_label'], not x['cluster_info']['is_cluster_head'])
    )

    return result

clustered_data = cluster_data(
    data_w_low_dim,
    coordinates_key='low_dim_embedding'
)
uuid_cluster = {e['uuid']: e['cluster_info']['cluster_label'] for e in clustered_data}
# pp(uuid_cluster)
# pp(clustered_data[:100])

In [7]:
def format_to_nested_clustering(clustered_data):
    """
    transform a list of object into a nested list based on clustering info.
    If there is only one cluster, it retruns the same input
    """
    # check the number of cluster heads; return if there is only one cluster
    num_cluster_heads = 0
    for item in clustered_data:
        if item['cluster_info']['is_cluster_head']:
            num_cluster_heads += 1
    if num_cluster_heads == 1:
        return clustered_data

    # Break down if there are more than one cluster
    result = []
    for item in clustered_data:
        item['children'] = item.get('children', [])
        if item['cluster_info']['is_cluster_head']:
            # Add cluster head to the tree and also add it as the first child
            result.append(item)
            if not item.get('children'):
                result[-1]['children'].append(deepcopy(item))
        else:
            result[-1]['children'].append(item)
    return result


In [29]:
def cluster_hierarchically(embedding_data_w_low_dim, include_original_cluster_label=False):
    """
    Gets an array of input data with dimension and performs
    clustering on them and represents data as hierarchical
    
    This function can be called recursively 
    
    Args
        [ {'low_dim_embedding': [], ...}, ...]
    """
    embedding_data_w_low_dim = deepcopy(embedding_data_w_low_dim)

    clustered_data = cluster_data(
        embedding_data_w_low_dim,
        coordinates_key='low_dim_embedding'
    )
    if include_original_cluster_label:
        for item in clustered_data:
            item['original_cluster_label'] = item['cluster_info']['cluster_label']

    nested_clusters = format_to_nested_clustering(clustered_data)
    return nested_clusters

nested_clusters = cluster_hierarchically(
    data_w_low_dim[:100],
    include_original_cluster_label=True
)
pp(nested_clusters)

[{'children': [{'children': [],
                'cluster_info': {'cluster_label': 0, 'is_cluster_head': True},
                'embedding': '',
                'low_dim_embedding': [9.501749, 22.677223],
                'original_cluster_label': 0,
                'text': 'Like most car detail enthusiasts, I never liked to '
                        'see 3 hours of detailing work destroyed in a couple '
                        'of days of driving and parking. With this brush, my '
                        'detail jobs last longer. The brush is gentle on the '
                        'clear coat but it does sweep the dust away, leaving '
                        'water and and bird droppings behind.For best results, '
                        'I recommend that you clay clean the car with a good '
                        'car clay and wax it with S100 car wax every few '
                        'months. Then using this brush, you can maintain your '
                        'car in show room 

                'low_dim_embedding': [-38.930172, 0.50636476],
                'original_cluster_label': 3,
                'text': 'This lock will only suffice to keep honest people '
                        'honest.  It is far too easy to just cut the cable.  '
                        'It is however a good deterrent.  I still would '
                        "recommend it just for it's deterrent aspect.",
                'uuid': 'd477166b-a7c9-4c7a-896a-e1133ed2f865'},
               {'children': [],
                'cluster_info': {'cluster_label': 3, 'is_cluster_head': False},
                'embedding': '',
                'low_dim_embedding': [-26.178783, 3.7430882],
                'original_cluster_label': 3,
                'text': 'This is just what I was looking for,  perfect size '
                        'for the spare on my 90 4runner.  Looped it through '
                        'the spaces in the wheel and locked in the back.  Nice '
                        'for the add

                'text': "So these aren't the best cables you can buy.  For "
                        "that you'll have to go to an electrical or welding "
                        'supply company and you will PAY!  Probably between 3 '
                        'and 5 times what these cost!  So unless you plan to '
                        'start a towing service and need cables that will see '
                        "daily use, I'd say these are gonna work for "
                        'you.(Update: After compairing them to the 4ga cables '
                        "I bought a month or so back for my car, they don't "
                        "really seem that much thicker...  That's said they "
                        'are MUCH heavier!  The individual stranding is of a '
                        "thicker gauge than the other model so I'm guessing "
                        "that's why the thickness isn't much more...)What I "
                        'like:--Length - I bought these for a ha

In [30]:
len(nested_clusters)

6

In [31]:
MAX_CLUSTER_SIZE = 10

# traverse tree and break down 
def bfs_break_down(head):
    """
    Traverse the nested clustering and break down if a node has too many 
    children
    """
    frontiers = [head]
    while frontiers:
        next = frontiers.pop(0)
        if len(next['children']) > MAX_CLUSTER_SIZE:
            next['children'] = cluster_hierarchically(next['children'])
        frontiers.extend(next['children'])
    

head = {}
head['children'] = nested_clusters
# pp(head)

bfs_break_down(head)

pp(head['children'])

[{'children': [{'children': [],
                'cluster_info': {'cluster_label': 0, 'is_cluster_head': True},
                'embedding': '',
                'low_dim_embedding': [9.501749, 22.677223],
                'original_cluster_label': 0,
                'text': 'Like most car detail enthusiasts, I never liked to '
                        'see 3 hours of detailing work destroyed in a couple '
                        'of days of driving and parking. With this brush, my '
                        'detail jobs last longer. The brush is gentle on the '
                        'clear coat but it does sweep the dust away, leaving '
                        'water and and bird droppings behind.For best results, '
                        'I recommend that you clay clean the car with a good '
                        'car clay and wax it with S100 car wax every few '
                        'months. Then using this brush, you can maintain your '
                        'car in show room 

                                      '$1750), with a Smitybilt tire carrier, '
                                      'the 36&#34; cable was plenty long '
                                      'enough.',
                              'uuid': 'eefbeb36-beb2-4d43-8e42-50c68fbec229'},
                             {'children': [],
                              'cluster_info': {'cluster_label': 0,
                                               'is_cluster_head': False},
                              'embedding': '',
                              'low_dim_embedding': [-40.189045, 1.2361538],
                              'original_cluster_label': 3,
                              'text': 'This lock appears to be nearly '
                                      'identical to the old masterlock '
                                      'design.  That one built up rust inside '
                                      'the lock mechanism, so this will '
                                      "probably d

                        'determine the appropriate line size.  Ambient '
                        'temperature, conductor, line distance, voltage, and '
                        'amperage.  Ambient temp we have no control over.  Hot '
                        'is bad so just assume the worst.  This means bigger '
                        'cable to overcome the additon resistance heat is '
                        "producing.  Conductor of these is copper so that's "
                        'good.  You want pure or at least a high purity mix of '
                        'copper in this type of conductor.  Distance and '
                        "voltage.  One of these cable's strengths also happens "
                        'to be a weakness.  Total run length for this circut '
                        "is 50'.  Pretty long for a low voltage system like "
                        '12VDC.  The higher the voltage, the longer it can run '
                        'on a given gauge wire without enco

In [32]:
def insert_children_count(head):
    """
    Add total number of children for each node recursively
    """
    if not head['children']:
        head['children_count'] = 0
        return 0
    sum = 0
    for node in head['children']:
        sum += 1 + node.get('children_count', insert_children_count(node))
    head['children_count'] = sum
    return sum

insert_children_count(head)
print(head['children_count'])
pp(head)

110
{'children': [{'children': [{'children': [],
                             'children_count': 0,
                             'cluster_info': {'cluster_label': 0,
                                              'is_cluster_head': True},
                             'embedding': '',
                             'low_dim_embedding': [9.501749, 22.677223],
                             'original_cluster_label': 0,
                             'text': 'Like most car detail enthusiasts, I '
                                     'never liked to see 3 hours of detailing '
                                     'work destroyed in a couple of days of '
                                     'driving and parking. With this brush, my '
                                     'detail jobs last longer. The brush is '
                                     'gentle on the clear coat but it does '
                                     'sweep the dust away, leaving water and '
                                     

                                                   'stepped on them from '
                                                   'time-to-time and have '
                                                   'never broken them. '
                                                   'However, I did break one '
                                                   'recently when I moved my '
                                                   'sofa - not the deflector '
                                                   'itself but one of those '
                                                   'little tabs that hold it '
                                                   'together in the center, so '
                                                   "I'm back to buy another. I "
                                                   'wasted a lot of money '
                                                   'prior to purchasing these '
                                                   'unbreakable 

                                           'uuid': 'd86de6ec-ee83-4be2-ac8a-0e2c03462416'},
                                          {'children': [],
                                           'children_count': 0,
                                           'cluster_info': {'cluster_label': 0,
                                                            'is_cluster_head': False},
                                           'embedding': '',
                                           'low_dim_embedding': [-42.687794,
                                                                 1.0873545],
                                           'original_cluster_label': 3,
                                           'text': 'this is a very nice '
                                                   'stainless steel design.  '
                                                   'Mine was a bit longer than '
                                                   'necessary so I put a few '
                   

                                     'price of a set of cheap Booster/Jumper '
                                     'Cables in a brick and morter store, you '
                                     'can buy extra long and heavy duty '
                                     "jumpers!  First off, don't be the person "
                                     'that not only needs to ask a kind '
                                     'passer-by for a "jump" but also if they '
                                     "have jumper cables.  It's MUCH easier to "
                                     'get a jump start if you have your own '
                                     'cables.Next lets talk about sizing.  '
                                     'Having the longest cable possible is a '
                                     'major plus if your car is parked up '
                                     'against something like a pole or wall, '
                                     'or even parked on a one 

In [33]:
def insert_d3uuid(head):
    """
    Traverse the tree of data and insert a unique identifier for 
    each node that will be used for d3 distinctions later
    """
    if not head:
        return
    seen_uuids = set()
    frontiers = [head]
    while frontiers:
        next = frontiers.pop(0)
        next['d3uuid'] = str(uuid4())
        frontiers.extend(next['children'])
    

insert_d3uuid(head)

pp(head['children'])

[{'children': [{'children': [],
                'children_count': 0,
                'cluster_info': {'cluster_label': 0, 'is_cluster_head': True},
                'd3uuid': '71e49408-ac24-4a73-a259-4d68d97548b9',
                'embedding': '',
                'low_dim_embedding': [9.501749, 22.677223],
                'original_cluster_label': 0,
                'text': 'Like most car detail enthusiasts, I never liked to '
                        'see 3 hours of detailing work destroyed in a couple '
                        'of days of driving and parking. With this brush, my '
                        'detail jobs last longer. The brush is gentle on the '
                        'clear coat but it does sweep the dust away, leaving '
                        'water and and bird droppings behind.For best results, '
                        'I recommend that you clay clean the car with a good '
                        'car clay and wax it with S100 car wax every few '
                   

 {'children': [{'children': [{'children': [],
                              'children_count': 0,
                              'cluster_info': {'cluster_label': 0,
                                               'is_cluster_head': True},
                              'd3uuid': '05d520b1-b8ff-40d7-9d89-b78f88857b2f',
                              'embedding': '',
                              'low_dim_embedding': [-3.9351587, -5.875512],
                              'original_cluster_label': 2,
                              'text': 'this product does what it is supposed '
                                      'to do, but, I am not real happy with '
                                      'the quality of construction.',
                              'uuid': '4d07b1bb-5365-402f-87a0-304d5aeba7fb'}],
                'children_count': 1,
                'cluster_info': {'cluster_label': 0, 'is_cluster_head': True},
                'd3uuid': 'ca4f9543-3fab-43b5-b031-fcd1e2bc988e',
            

                'cluster_info': {'cluster_label': 0, 'is_cluster_head': False},
                'd3uuid': '281bf34a-9619-49cf-9a11-c6b403d07d76',
                'embedding': '',
                'low_dim_embedding': [-29.920412, -14.824564],
                'original_cluster_label': 4,
                'text': 'These long cables work fine for my truck, but the '
                        'quality seems a little on the shabby side. For the '
                        'money I was not expecting 200 dollar snap-on jumper '
                        'cables but these seem more like what you would see at '
                        'a chinese knock off shop like harbor freight for 30 '
                        'bucks.',
                'uuid': '747be46a-ea3a-4c11-a909-1cf824265def'},
               {'children': [],
                'children_count': 0,
                'cluster_info': {'cluster_label': 0, 'is_cluster_head': False},
                'd3uuid': '0dcfc994-8701-49b9-9c8e-2011132b7a07',
     

                        "which doesn't have a wiper, because it keeps the "
                        'window clearer. Follow the directions--let it dry to '
                        'a light haze and wipe it off with a microfiber or '
                        "paper towel until the windshield is clear. I'm sure "
                        "you'll like it.",
                'uuid': '6f681e46-3aa1-437d-a1fe-533a29782590'},
               {'children': [],
                'children_count': 0,
                'cluster_info': {'cluster_label': 0, 'is_cluster_head': False},
                'd3uuid': 'f06d7fb9-709e-44b8-8cda-28593766e70b',
                'embedding': '',
                'low_dim_embedding': [14.757531, 8.88684],
                'original_cluster_label': 5,
                'text': 'This is the original product.  Really great stuff.  '
                        "Works so well; when rain hits, you'll be so happy you "
                        "applied it.  Rain just sheets off; you don'

In [69]:
# calculate radius for each bubble 
MIN_RADIUS = 1
MIN_ALLOWED_DISTANCE = 1

def flatten_data(head):
    """
    Gets a tree of data and transform it into a flat list
    and removes repeated items. 
    """
    if not head:
        return
    ret_val = []
    seen_uuids = set()
    frontiers = [head]
    while frontiers:
        next = frontiers.pop(0)
        if next.get('uuid') and next['uuid'] not in seen_uuids:
            next_copy = deepcopy(next)
            next_copy.pop('children')
            ret_val.append(next_copy)
            seen_uuids.add(next['uuid'])                
        frontiers.extend(next['children'])
    return ret_val

def get_multiplier(clustering_data):
    """
    get the max multiplier that is used to inflate the bubble sizes
    Args
        clustering_data : list(dict) : a list of objects. Objects have a key for number of children
    """
    multiplier = float("inf")
    if len(clustering_data) < 2:
        multiplier = 0
    for i in range(len(clustering_data)):
        for j in range(i+1, len(clustering_data)):
            filled = max(
                np.sqrt(clustering_data[i]['children_count']),
                np.sqrt(clustering_data[j]['children_count'])
            )
            p1 = np.array([
                float(clustering_data[i]['low_dim_embedding'][0]),
                float(clustering_data[i]['low_dim_embedding'][1])
            ])
            p2 = np.array([
                float(clustering_data[j]['low_dim_embedding'][0]),
                float(clustering_data[j]['low_dim_embedding'][1])
            ])
            d = np.linalg.norm(p1 - p2)
            if d > MIN_ALLOWED_DISTANCE:
                multiplier = min(multiplier, d / filled)
#                 print(multiplier, d, filled, (
#                     clustering_data[i]['children_count'], clustering_data[j]['children_count']
#                 ))
    return multiplier


# clustering_data = flatten_data(head)
# pp(clustering_data)

radius_multiplier_factor = get_multiplier(head['children'])
print(radius_multiplier_factor)

def insert_radius(head, radius_multiplier_factor):
    """
    Insert the radius in all object in the tree, and also for
    each object, insert the radius of their parent
    """
    frontiers = [head]
    while frontiers:
        next = frontiers.pop(0)
        if next['children']:
            next['radius'] = max([
                np.sqrt(next['children_count']) * radius_multiplier_factor,
                MIN_RADIUS
            ])
        else:
            next['radius'] = MIN_RADIUS            
        frontiers.extend(next['children'])

                
insert_radius(head, radius_multiplier_factor)
pp(head)

2.3317806301257535
{'children': [{'children': [{'children': [],
                             'children_count': 0,
                             'cluster_info': {'cluster_label': 0,
                                              'is_cluster_head': True},
                             'd3uuid': '71e49408-ac24-4a73-a259-4d68d97548b9',
                             'embedding': '',
                             'low_dim_embedding': [9.501749, 22.677223],
                             'original_cluster_label': 0,
                             'parent': {'low_dim_embedding': [9.501749,
                                                              22.677223],
                                        'radius': 5.950951240271025},
                             'radius': 1,
                             'text': 'Like most car detail enthusiasts, I '
                                     'never liked to see 3 hours of detailing '
                                     'work destroyed in a couple of days of '


               'original_cluster_label': 0,
               'parent': {'low_dim_embedding': None,
                          'radius': 12.482820631650778},
               'radius': 11.658903150628767,
               'text': 'Like most car detail enthusiasts, I never liked to see '
                       '3 hours of detailing work destroyed in a couple of '
                       'days of driving and parking. With this brush, my '
                       'detail jobs last longer. The brush is gentle on the '
                       'clear coat but it does sweep the dust away, leaving '
                       'water and and bird droppings behind.For best results, '
                       'I recommend that you clay clean the car with a good '
                       'car clay and wax it with S100 car wax every few '
                       'months. Then using this brush, you can maintain your '
                       'car in show room condition for a long time without '
                       '

                                                      'radius': 3.947414481935819},
                                           'radius': 1,
                                           'text': 'I ordered two models of '
                                                   'Deflecto vents and both '
                                                   'are outstanding.  Heavy '
                                                   'duty plastic, large strong '
                                                   'magnets on the side and '
                                                   'they work perfectly.  '
                                                   'Would recommend to anyone, '
                                                   'especially since these are '
                                                   'impossible to find '
                                                   'locally, just try asking '
                                                   'for one in your local hom

                                     'and I have no shore power, solar, or '
                                     'generator to recharge.Bought the bucket '
                                     'boss 06009 jumper cable bag and it fit '
                                     'this 25 footer Perfectly!!!It has no use '
                                     'and is a waste of money right now ... '
                                     'but will EASILY pay for itself the first '
                                     'time you need it ... always be prepared! '
                                     ':)',
                             'uuid': 'a37b007e-8461-46d5-b1ce-9f6103a645cd'},
                            {'children': [],
                             'children_count': 0,
                             'cluster_info': {'cluster_label': 0,
                                              'is_cluster_head': False},
                             'd3uuid': '718afe2e-2d23-4d63-9ba4-d039b6c774f2',
         

                             'children_count': 0,
                             'cluster_info': {'cluster_label': 0,
                                              'is_cluster_head': False},
                             'd3uuid': '6d6bf4fa-2729-4bfb-8c77-e4ab22e350bd',
                             'embedding': '',
                             'low_dim_embedding': [-11.938072, 8.6725],
                             'original_cluster_label': 5,
                             'parent': {'low_dim_embedding': [9.224624,
                                                              11.021614],
                                        'radius': 5.707951910357741},
                             'radius': 1,
                             'text': 'All other jumper cables are not real '
                                     'jumper cables.  THESE are jumper '
                                     "cables.  Let me explain.When you're in a "
                                     'bind on some dark night in so

In [76]:
def insert_parents_info(head):
    """
    Insert parents coordinates and radius in each child
    """
    if not head:
        return
    frontiers = [head]
    seen = []
    while frontiers:
        next = frontiers.pop(0)
        for child in next.get('children', []):
            child['parent'] = {
                'low_dim_embedding': next.get('low_dim_embedding'),
                'radius': next.get('radius'),
                'd3uuid': next.get('d3uuid')
            }
        frontiers.extend(next['children'])
    return

insert_parents_info(head)
pp(head)

{'children': [{'children': [{'children': [],
                             'children_count': 0,
                             'cluster_info': {'cluster_label': 0,
                                              'is_cluster_head': True},
                             'd3uuid': '71e49408-ac24-4a73-a259-4d68d97548b9',
                             'embedding': '',
                             'low_dim_embedding': [9.501749, 22.677223],
                             'original_cluster_label': 0,
                             'parent': {'d3uuid': 'b3c48ead-2bfe-4062-b449-2455682a8309',
                                        'low_dim_embedding': [9.501749,
                                                              22.677223],
                                        'radius': 11.658903150628767},
                             'radius': 1,
                             'text': 'Like most car detail enthusiasts, I '
                                     'never liked to see 3 hours of detailing '
      

                            {'children': [],
                             'children_count': 0,
                             'cluster_info': {'cluster_label': 0,
                                              'is_cluster_head': False},
                             'd3uuid': '092708b7-ba2f-4e9e-b9c7-00d37a345b53',
                             'embedding': '',
                             'low_dim_embedding': [14.617445, 21.852104],
                             'original_cluster_label': 0,
                             'parent': {'d3uuid': 'b3c48ead-2bfe-4062-b449-2455682a8309',
                                        'low_dim_embedding': [9.501749,
                                                              22.677223],
                                        'radius': 11.658903150628767},
                             'radius': 1,
                             'text': 'The mk4 Beetles have a dashboard made of '
                                     'a ridiculous rubber/plastic material '
  

                                                   'perfectly on my ceiling '
                                                   'vents and the diffusers '
                                                   'appear pretty sturdy.  So '
                                                   'far, nothing has fallen to '
                                                   '"crack me in the head."The '
                                                   'diffusers are adjustable '
                                                   'so that you can get them '
                                                   'to fit even if you have a '
                                                   'vent lever or screws that '
                                                   'may get in the way, which '
                                                   'I do.I am very pleased and '
                                                   'would recommend this '
                                                

                             'low_dim_embedding': [-38.930172, 0.50636476],
                             'original_cluster_label': 3,
                             'parent': {'d3uuid': 'dd3b8049-42a7-4f67-a2fa-4df82c3e1ad7',
                                        'low_dim_embedding': [-37.461365,
                                                              0.05802356],
                                        'radius': 8.724724219046422},
                             'radius': 7.73364144354561,
                             'text': 'This lock will only suffice to keep '
                                     'honest people honest.  It is far too '
                                     'easy to just cut the cable.  It is '
                                     'however a good deterrent.  I still would '
                                     "recommend it just for it's deterrent "
                                     'aspect.',
                             'uuid': 'd477166b-a7c9-4c7a-896a-e113

                                        'radius': 10.937020615992843},
                             'radius': 1,
                             'text': 'This works perfectly and sets my mind at '
                                     'ease with all the light fingered '
                                     'individuals out there ready to lift your '
                                     'battery, propane tank and whatever else '
                                     'they can get their hands on..',
                             'uuid': '6baa3832-b365-4d54-956a-7c19c2a80331'}],
               'children_count': 22,
               'cluster_info': {'cluster_label': 4, 'is_cluster_head': True},
               'd3uuid': '71b73c52-5e3b-44ba-8e9a-573de5ad37ce',
               'embedding': '',
               'low_dim_embedding': [-24.88468, -9.391542],
               'original_cluster_label': 4,
               'parent': {'d3uuid': '8b704066-5bf5-4bd8-9a42-b58c4cbd3c59',
                          'low_

                             'original_cluster_label': 5,
                             'parent': {'d3uuid': 'ff5d467c-f950-4044-93b9-5c2434f75c26',
                                        'low_dim_embedding': [9.224624,
                                                              11.021614],
                                        'radius': 11.182827051407084},
                             'radius': 1,
                             'text': 'Rain-X 800002243 Glass Treatment- 7 '
                                     "oz.It's simple, this stuff works great "
                                     'when applied correctly, the rain will '
                                     'slide right off!',
                             'uuid': '54464830-7e15-4848-9937-36442e56d886'},
                            {'children': [],
                             'children_count': 0,
                             'cluster_info': {'cluster_label': 0,
                                              'is_cluster_head':

In [77]:
def insert_meta_data(head):
    """
    Insert the metadat field that includes the following
        metadata:
            max_x, min_x
            max_y, min_y
    """
    min_x = float("inf")
    max_x = float("-inf")
    min_y = float("inf")
    max_y = float("-inf")
    frontiers = deepcopy(head['children'])
    while frontiers:
        next = frontiers.pop(0)
        min_x = min(next['low_dim_embedding'][0] - next['radius'], min_x)
        min_y = min(next['low_dim_embedding'][1] - next['radius'], min_y)
        max_x = max(next['low_dim_embedding'][0] + next['radius'], max_x)
        max_y = max(next['low_dim_embedding'][1] + next['radius'], max_y)
        frontiers.extend(next['children'])
    head['metadata'] = {
        'x': {
            'max': max_x,
            'min': min_x,
        },
        'y': {
            'max': max_y,
            'min': min_y,
        },
        'radius': {
            'max': max(max_x - min_x, max_y - min_y)
        }
    }

insert_meta_data(head)

In [78]:
pp(head)

{'children': [{'children': [{'children': [],
                             'children_count': 0,
                             'cluster_info': {'cluster_label': 0,
                                              'is_cluster_head': True},
                             'd3uuid': '71e49408-ac24-4a73-a259-4d68d97548b9',
                             'embedding': '',
                             'low_dim_embedding': [9.501749, 22.677223],
                             'original_cluster_label': 0,
                             'parent': {'d3uuid': 'b3c48ead-2bfe-4062-b449-2455682a8309',
                                        'low_dim_embedding': [9.501749,
                                                              22.677223],
                                        'radius': 11.658903150628767},
                             'radius': 1,
                             'text': 'Like most car detail enthusiasts, I '
                                     'never liked to see 3 hours of detailing '
      

                                        'radius': 11.658903150628767},
                             'radius': 1,
                             'text': 'Bought this for my 2013 Evo x gsr. It '
                                     'left red lint all over my dash. If using '
                                     "in the house I'm sure it's fine as you "
                                     "won't notice. But this brush DOES leave "
                                     'red fibers behind',
                             'uuid': '98f4631f-c12a-4638-a834-4fc21c64b53c'},
                            {'children': [],
                             'children_count': 0,
                             'cluster_info': {'cluster_label': 0,
                                              'is_cluster_head': False},
                             'd3uuid': '8e5acd40-57a0-455f-902d-d8139d2322e8',
                             'embedding': '',
                             'low_dim_embedding': [7.466486, 31.27101],
  

                                                   'entire metal register out '
                                                   'of the floor. The magnets '
                                                   'are strong enough to hold '
                                                   'tight on registers that '
                                                   'have several coats of '
                                                   'paint on them, too.',
                                           'uuid': '07fdb0bd-b2a6-4025-befa-dc0d7b436e7e'},
                                          {'children': [],
                                           'children_count': 0,
                                           'cluster_info': {'cluster_label': 0,
                                                            'is_cluster_head': False},
                                           'd3uuid': '5356660f-5823-411b-8628-c4607459f730',
                                           'embedding': '

                                           'low_dim_embedding': [-36.599335,
                                                                 -0.8629264],
                                           'original_cluster_label': 3,
                                           'parent': {'d3uuid': '59e6cb81-8fd9-443d-a27c-64f76052a3f6',
                                                      'low_dim_embedding': [-38.930172,
                                                                            0.50636476],
                                                      'radius': 7.73364144354561},
                                           'radius': 1,
                                           'text': 'It is perfect to secure '
                                                   'your spare tire to tire '
                                                   'carrier or bumper. Easy to '
                                                   'use and sturdy. Is not a '
                                     

                             'd3uuid': '1436556a-6112-46a9-8ce3-da18296cf259',
                             'embedding': '',
                             'low_dim_embedding': [-30.415787, -13.532351],
                             'original_cluster_label': 4,
                             'parent': {'d3uuid': '71b73c52-5e3b-44ba-8e9a-573de5ad37ce',
                                        'low_dim_embedding': [-24.88468,
                                                              -9.391542],
                                        'radius': 10.937020615992843},
                             'radius': 1,
                             'text': "I'm one of those guys who stops and "
                                     'helps people when they need it.  While '
                                     'not a pro recovery guy I use my cables '
                                     'more than the average person.  These fit '
                                     'the bill just right. Strong clamps wit

                             'radius': 1,
                             'text': 'People were complaining about streaks '
                                     'but this works just as good as my 10 '
                                     'year old model it replaced. The handle '
                                     "feels a little cheaper but I haven't had "
                                     'any issues. Then again my car is white '
                                     'and maybe any streaking is harder to '
                                     'see. Just be sure to leave it out on '
                                     'some newspaper for a few days for best '
                                     'results.',
                             'uuid': '5995ce3f-050d-4158-847f-cd4acf241788'},
                            {'children': [],
                             'children_count': 0,
                             'cluster_info': {'cluster_label': 0,
                                           

                             'parent': {'d3uuid': 'ff5d467c-f950-4044-93b9-5c2434f75c26',
                                        'low_dim_embedding': [9.224624,
                                                              11.021614],
                                        'radius': 11.182827051407084},
                             'radius': 1,
                             'text': 'Rain-X 800002243 Glass Treatment- 7 '
                                     "oz.It's simple, this stuff works great "
                                     'when applied correctly, the rain will '
                                     'slide right off!',
                             'uuid': '54464830-7e15-4848-9937-36442e56d886'},
                            {'children': [],
                             'children_count': 0,
                             'cluster_info': {'cluster_label': 0,
                                              'is_cluster_head': False},
                             'd3uuid': '64b451f2-

In [81]:
def get_formatted_item(item):
    """
    Arg:
        An input item
    """
    entry = {
        'x': float(item['low_dim_embedding'][0]),
        'y': float(item['low_dim_embedding'][1]),
        'uuid': item.get('uuid'),
        'd3uuid': item.get('d3uuid'),
        'text': item.get('text'),
        'cluster_label': int(item['original_cluster_label']),
        'children_count': item['children_count'],
        'radius': item['radius'],
        'parent': {
            'x': float(item['parent']['low_dim_embedding'][0]) if item['parent']['low_dim_embedding'] else 0,
            'y': float(item['parent']['low_dim_embedding'][1]) if item['parent']['low_dim_embedding'] else 0,
            'radius': float(item['parent']['radius']),
            'd3uuid': item['parent']['d3uuid']
        }
    }
    return entry


def get_formatted_data(node):
    """
    
    """
    if not node:
        return
    new_node = {}
    if 'low_dim_embedding' in node:
        new_node = get_formatted_item(node)
    new_node['children'] = [
        get_formatted_data(c) for c in node['children']
    ]
    return new_node

#
input_data = deepcopy(head)
# pp(input_data)
formatted_data = get_formatted_data(input_data)
formatted_data['metadata'] = head['metadata']

In [82]:
pp(formatted_data)

{'children': [{'children': [{'children': [],
                             'children_count': 0,
                             'cluster_label': 0,
                             'd3uuid': '71e49408-ac24-4a73-a259-4d68d97548b9',
                             'parent': {'d3uuid': 'b3c48ead-2bfe-4062-b449-2455682a8309',
                                        'radius': 11.658903150628767,
                                        'x': 9.501749038696289,
                                        'y': 22.677223205566406},
                             'radius': 1,
                             'text': 'Like most car detail enthusiasts, I '
                                     'never liked to see 3 hours of detailing '
                                     'work destroyed in a couple of days of '
                                     'driving and parking. With this brush, my '
                                     'detail jobs last longer. The brush is '
                                     'gentle on the 

                             'y': 9.494244575500488},
                            {'children': [],
                             'children_count': 0,
                             'cluster_label': 1,
                             'd3uuid': 'aa5a8015-20a5-4a90-91be-c83f728afbf8',
                             'parent': {'d3uuid': 'cce6842b-3e3a-493e-9e3b-41908bf048b4',
                                        'radius': 5.711672735913528,
                                        'x': 37.470115661621094,
                                        'y': 8.433188438415527},
                             'radius': 1,
                             'text': 'Have you ever felt like your auto '
                                     'mechanic was "Mr. Wizard"?  All the way '
                                     'down to the bad attitude?  Only to find '
                                     'he has a doo-dad sitting somewhere out '
                                     'of eyesight that tells all, like a '
    

                                           'children_count': 0,
                                           'cluster_label': 3,
                                           'd3uuid': '5315af44-4b96-4639-80a6-cfce9b189ee0',
                                           'parent': {'d3uuid': '59e6cb81-8fd9-443d-a27c-64f76052a3f6',
                                                      'radius': 7.73364144354561,
                                                      'x': -38.930171966552734,
                                                      'y': 0.5063647627830505},
                                           'radius': 1,
                                           'text': 'This lock appears to be '
                                                   'nearly identical to the '
                                                   'old masterlock design.  '
                                                   'That one built up rust '
                                                   'inside the lock

                             'x': -23.807392120361328,
                             'y': -7.753734588623047},
                            {'children': [],
                             'children_count': 0,
                             'cluster_label': 4,
                             'd3uuid': 'a07788df-b195-45a2-8ba2-59a6f0a47fce',
                             'parent': {'d3uuid': '71b73c52-5e3b-44ba-8e9a-573de5ad37ce',
                                        'radius': 10.937020615992843,
                                        'x': -24.884679794311523,
                                        'y': -9.391542434692383},
                             'radius': 1,
                             'text': 'This was exactly what I needed for '
                                     'transporting my fishing kayak in the bed '
                                     'of my F-150. Easy to assemble and works '
                                     'great!',
                             'uuid': 'ade2b9de-bb7

                                     'wipers seemed to even it out though.',
                             'uuid': 'a97160c9-c941-43c4-8289-cf8a120ea71d',
                             'x': 12.307098388671875,
                             'y': 10.416742324829102},
                            {'children': [],
                             'children_count': 0,
                             'cluster_label': 5,
                             'd3uuid': '6bbd7c9c-568e-409a-b68f-c8061c9c9c2b',
                             'parent': {'d3uuid': 'ff5d467c-f950-4044-93b9-5c2434f75c26',
                                        'radius': 11.182827051407084,
                                        'x': 9.224623680114746,
                                        'y': 11.021614074707031},
                             'radius': 1,
                             'text': 'great stuff to have when it starts to '
                                     "rain.  Don't need it too often in So "
                           

In [83]:
lines = json.dumps(formatted_data, indent=4)
with open('all_levels.json', 'w') as f:
    f.write(lines)

In [ ]:
# get level n of the tree
def get_nth_level_nodes(head, n):
    level = 0
    result = []
    frontiers = [head, None]
    while frontiers and frontiers != [None]:
        next = frontiers.pop(0)
        if next is None:
            if level == n:
                return result
            result = [e for e in result if not e['children']]
            level += 1
            frontiers.append(None)
            continue
        result.append(next)
        frontiers.extend(next['children'])
    if level < n:
        return
    return result

# pp(get_nth_level_nodes(head, 3))
            

In [ ]:
all_levels_raw = {0: ['zero level']}
head
level = 0
while all_levels_raw[level]:
    level += 1
    all_levels_raw[level] = get_nth_level_nodes(head, level)

In [ ]:
pp(all_levels_raw)

In [ ]:
def get_formatted_data(data):
    """
    
    """
    metadata = []
    if not data:
        return 
    for item in data:
        parent_xy = item['parent']['low_dim_embedding']
        if not parent_xy:
            parent_xy = item['low_dim_embedding']
        parent_r = item['parent']['radius']
        if not parent_r:
            parent_r = item['radius']
        entry = {
            'x': float(item['low_dim_embedding'][0]),
            'y': float(item['low_dim_embedding'][1]),
            'uuid': item.get('uuid'),
            'text': item.get('text'),
            'cluster_label': int(item['original_cluster_label']),
            'children_count': item['children_count'],
            'radius': item['radius'],
            'parent': {
                'x': float(parent_xy[0]),
                'y': float(parent_xy[1]),
                'radius': parent_r,
            }
        }
        metadata.append(entry)
    return metadata

#
all_levels = {}
level = 1
while all_levels_raw[level]:
    all_levels[level] = get_formatted_data(all_levels_raw[level])
    level += 1

pp(all_levels)

In [ ]:
for l in all_levels:
    print(len(all_levels[l]))

In [ ]:
last_level = len(all_levels)
while not all_levels[last_level] and last_level > 0:
    last_level -= 1
all_x = [item['x'] for item in all_levels[last_level]]
all_y = [item['y'] for item in all_levels[last_level]]
x_range = [min(all_x), max(all_x)]
y_range = [min(all_y), max(all_y)]
print(x_range, y_range)

In [ ]:
lines = json.dumps(all_levels, indent=4)
with open('all_levels.json', 'w') as f:
    f.write(lines)

In [ ]:
dummy_data = [
    {
        'uuid': 'uuid-001',
        'x': '1', 
        'y': '2',
        'text': 'test',
        'is_cluster_head': True
    },
    {
        'uuid': 'uuid-002',
        'x': '1', 
        'y': '2',
        'text': 'test',
        'is_cluster_head': False
    },
    {
        'uuid': 'uuid-003',
        'x': '1', 
        'y': '2',
        'text': 'test',
        'is_cluster_head': False
    },
    {
        'uuid': 'uuid-004',
        'x': '1', 
        'y': '2',
        'text': 'test',
        'is_cluster_head': False
    },
    {
        'uuid': 'uuid-005',
        'x': '1', 
        'y': '2',
        'text': 'test',
        'is_cluster_head': True
    },
    {
        'uuid': 'uuid-006',
        'x': '1', 
        'y': '2',
        'text': 'test',
        'is_cluster_head': False
    },
]

In [ ]:
def get_nested_texts(clustred_texts):
    """
    re-formats the flat structure for clustered texts
    to a hierarchical structure
    """
    nested_data = []
    for item in clustred_texts:
        item['children'] = item.get('children', [])
        if item.get('is_cluster_head'):
            nested_data.append(formatted_item)
        else:
            nested_data[-1]['children'].append(formatted_item)
    return nested_data


nested_data = get_nested_texts(data_summary)
# nested_data = get_nested_texts(dummy_data)
import pprint
pprint.PrettyPrinter().pprint(nested_data)

In [ ]:
total_count = len(cluster_labels)
from collections import defaultdict
class_count = defaultdict(int)
for label in cluster_labels:
    class_count[label] += 1
class_count